<a href="https://colab.research.google.com/github/prof-atritiack/visao-computacional-huggingface/blob/main/huggingface_visao_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Hugging Face - Visão Computacional (Colab com Visualização)**

Este notebook apresenta um fluxo **didático e direto** para trabalhar com modelos de **classificação** e **detecção de objetos** do Hugging Face.  
O padrão de exibição de imagens segue o mesmo usado no notebook do **YOLO**, com `cv2_imshow`.  


## **1. Instalação das dependências**

In [ ]:
!pip install -U huggingface_hub transformers pillow torch

In [ ]:
from warnings import filterwarnings
filterwarnings('ignore')

## **2. Login no Hugging Face Hub**
Use o token salvo nos *secrets* do Colab (`HF_TOKEN`).

In [ ]:
from huggingface_hub import login
from google.colab import userdata

token = userdata.get('HF_TOKEN')  # defina o token nos secrets do Colab
login(token=token)

## **3. Preparação das imagens de teste**
As imagens devem ser colocadas na pasta `datasets/imagens/`. Será usada a imagem `exemplo.jpg`.

In [ ]:
from pathlib import Path
from PIL import Image

IMAGES_DIR = Path("datasets/imagens")
IMAGES_DIR.mkdir(parents=True, exist_ok=True)

exemplo_imagem = IMAGES_DIR / "exemplo.jpg"

if exemplo_imagem.exists():
    print("Imagem encontrada:", exemplo_imagem)
else:
    print("Adicione uma imagem chamada 'exemplo.jpg' em datasets/imagens/")

## **4. Classificação de imagens**
Será usado o modelo `google/vit-base-patch16-224`. O resultado mostrará a imagem e os rótulos previstos.

In [ ]:
from transformers import pipeline
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

In [ ]:
MODEL_CLF = 'google/vit-base-patch16-224'
clf = pipeline(task='image-classification', model=MODEL_CLF)

In [ ]:
if exemplo_imagem.exists():
    img = Image.open(exemplo_imagem).convert('RGB')
    preds = clf(str(exemplo_imagem))
    print('Resultados da classificação:')
    for p in preds:
        print(f"{p['label']}: {100*(p['score']):.2f} %")
    # Exibir imagem
    # img_cv = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
    # cv2_imshow(img_cv)

## **5. Detecção de objetos**
Será usado o modelo `facebook/detr-resnet-50`. As caixas serão desenhadas na imagem e exibidas no Colab.

In [ ]:
from PIL import ImageDraw

In [ ]:
MODEL_DET = 'facebook/detr-resnet-50'
detector = pipeline(task='object-detection', model=MODEL_DET)

In [ ]:
if exemplo_imagem.exists():
    img = Image.open(exemplo_imagem).convert('RGB')
    draw = ImageDraw.Draw(img)
    dets = detector(str(exemplo_imagem))
    print('Resultados da detecção:')
    for d in dets:
        box = d['box']
        draw.rectangle([(box['xmin'], box['ymin']), (box['xmax'], box['ymax'])], outline='red', width=3)
        draw.text((box['xmin'], box['ymin']-10), f"{d['label']} {d['score']:.2f}", fill='red')
        print(f"{d['label']} ({100*(d['score']):.2f} %) -> {box}")

In [ ]:
# Exibir imagem processada no Colab
img_cv = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
cv2_imshow(img_cv)

## **6. Uso da API HTTP do Hugging Face**
Exemplo de como enviar a imagem para a API de inferência do Hugging Face.

In [ ]:
import requests, json
from google.colab import userdata

In [ ]:
# token do Colab Secrets
HF_TOKEN = userdata.get('HF_TOKEN')
if HF_TOKEN is None:
    HF_TOKEN = input("⚠️ Token Hugging Face não encontrado nos secrets. Cole seu HF_TOKEN: ")

headers = {
    "Authorization": f"Bearer {HF_TOKEN}",
    "Content-Type": "application/octet-stream"
}

In [ ]:
API_MODEL = MODEL_CLF
API_URL = f"https://api-inference.huggingface.co/models/{API_MODEL}"

In [ ]:
if exemplo_imagem.exists():
    with open(exemplo_imagem, "rb") as f:
        data = f.read()
    resp = requests.post(API_URL, headers=headers, data=data)

    print("Status da requisição:", resp.status_code)
    try:
        resposta = resp.json()
        print("Resposta da API:")
        print(json.dumps(resposta, indent=2, ensure_ascii=False))
    except Exception as e:
        print("Erro ao interpretar resposta:", e)
        print(resp.text)
else:
    print("Adicione a imagem exemplo.jpg em datasets/imagens/")

In [ ]:
import json
from PIL import Image, ImageDraw

In [ ]:
exemplo_imagem = IMAGES_DIR / "person.jpg"  # troque para o nome do arquivo desejado

In [ ]:
if exemplo_imagem.exists():
    img = Image.open(exemplo_imagem).convert("RGB")
    print("Imagem carregada:", exemplo_imagem)

    # ==== 2. Usar a API HTTP do Hugging Face para detecção ====
    API_MODEL = MODEL_DET  # definido antes (facebook/detr-resnet-50)
    API_URL = f"https://api-inference.huggingface.co/models/{API_MODEL}"

    with open(exemplo_imagem, "rb") as f:
        data = f.read()
    resp = requests.post(
        API_URL,
        headers={**headers, "Content-Type": "image/jpeg"},
        data=data
    )

    print("Status da requisição:", resp.status_code)
    resposta = resp.json()
    print("Resposta bruta da API (Detecção):")
    print(json.dumps(resposta, indent=2, ensure_ascii=False))

    # ==== 3. Desenhar bounding boxes na imagem com cores variadas ====
    draw = ImageDraw.Draw(img)
    cores = ["red", "blue", "green", "orange", "purple", "cyan", "magenta", "yellow"]

    if isinstance(resposta, list):
        for i, d in enumerate(resposta):
            if "box" in d:
                box = d["box"]
                cor = cores[i % len(cores)]  # alterna entre as cores da lista
                draw.rectangle(
                    [(box["xmin"], box["ymin"]), (box["xmax"], box["ymax"])],
                    outline=cor, width=3
                )
                draw.text(
                    (box["xmin"], box["ymin"]-10),
                    f"{d['label']} {d['score']:.2f}",
                    fill=cor
                )
    else:
        print("⚠️ A resposta não contém uma lista de detecções.")

    # ==== 4. Exibir imagem com bounding boxes no Colab ====
    img_cv = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
    cv2_imshow(img_cv)
else:
    print("⚠️ Imagem não encontrada:", exemplo_imagem)
